# Two-box model estimating the effects of touristic activities on lakes

This script runs a two-box (epilimnion-hypolimnion) model of a lake to estimate the effects of touristic activities. 

**Author**: Tomy Doda, tomy.doda@unil.ch \
**Last update**: dd.mm.2025

### Load the packages

In [14]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'svg.fonttype':'none', 'font.sans-serif':'Arial','font.size': 12}) # "none": to export text as text
import pandas as pd
from datetime import datetime, timezone, UTC, timedelta
import math
import cmocean
from functions import load_parameters
from compound import COMP
# Uncomment to have interactive figures:
%matplotlib widget 

***

### Options (to change for different simulations)

Specify the lake name, according to the files in the `lakes` folder:

In [15]:
lakename="muzelle"

Specify the processes to include among the following options, according to the files in the `processes` folder:
* sunscreen

In [16]:
processes=["sunscreen"]

Specify the activities to include among the following options, according to the files in the `activities` folder:
* swimming

In [17]:
activities=["swimming"]

Figure saving option:

In [18]:
savefig=False # If True, all figures are exported as png

***

### Load the parameters from csv files

Lake-specific parameters:

In [19]:
param_lake,_,_=load_parameters(os.path.join("lakes",lakename+".csv"))

Process-specific parameters:

In [20]:
param_process=dict()
units_process=dict()
for p in processes:
    param_process[p],units_process[p],_=load_parameters(os.path.join("processes",p+".csv"))

Activity-specific parameters:

In [21]:
param_act=dict()
for a in activities:
    param_act[a],_,_=load_parameters(os.path.join("activities",a+".csv"))

Simulation parameters:

In [22]:
param_sim,_,_=load_parameters(os.path.join("simulation","simulation.csv"))

### Create variables

Time vector:

In [23]:
tdate=np.arange(param_sim["Start_date"],param_sim["End_date"],timedelta(hours=param_sim["Time_step"]))
tnum=tdate.astype("datetime64[ns]").astype(np.float64)*1e-9

Compound objects:

In [24]:
modelled_compounds=dict()
for key,value in param_process.items():
    modelled_compounds[key]=COMP(name_comp=value["Name_compound"],conc_units=units_process[key]["Initial_concentration"])
    modelled_compounds[key].set_time_series(param_sim,value)

### Modelling

In [25]:
modelled_compounds["sunscreen"].data

{'time': array([1.7487360e+09, 1.7487396e+09, 1.7487432e+09, ..., 1.7592660e+09,
        1.7592696e+09, 1.7592732e+09]),
 'tdate': array(['2025-06-01T00:00:00.000000', '2025-06-01T01:00:00.000000',
        '2025-06-01T02:00:00.000000', ..., '2025-09-30T21:00:00.000000',
        '2025-09-30T22:00:00.000000', '2025-09-30T23:00:00.000000'],
       dtype='datetime64[us]'),
 'conc': array([ 0., nan, nan, ..., nan, nan, nan])}

Create modelling PDE as a function of class "compound" (iterative loop)